<a href="https://colab.research.google.com/github/stevecwb/Modelando-a-epidemia-de-Covid-19/blob/main/Modelando_a_epidemia_de_Covid_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelando a epidemia de Covid-19
 
Trabalho de MS211 (Cálculo Numérico), segundo semestre de 2021.

## Grupo X:
### Daniel C. Neto RA: 169233
### Estevão S. Rodrigues RA: 168030
### Nicole M. Baranzano RA: 186201

# Modelo SEIR

O modelo SEIR consiste em resolver as seguintes equações diferenciais [[1](#referencia)]:
$$\begin{aligned}
\frac{d \mathcal{S}}{d t} &=-\frac{r_{0}}{T_{\mathrm{inf}}} \mathcal{S} \mathcal{I} \\
\frac{d \mathcal{E}}{d t} &=\frac{r_{0}}{T_{\mathrm{inf}}} \mathcal{S} \mathcal{I}-\frac{1}{T_{\mathrm{inc}}} \mathcal{E} \\
\frac{d \mathcal{I}}{d t} &=\frac{1}{T_{\mathrm{inc}}} \mathcal{E}-\frac{1}{T_{\mathrm{inf}}} \mathcal{I} \\
\frac{d \mathcal{R}}{d t} &=\frac{1}{T_{\mathrm{inf}}} \mathcal{I}
\end{aligned}$$

Sabendo que, segundo os dados do enunciado, tem-se os seguintes valores:
- Número de reprodução basal: $r_0=2,5$ [[2](#referencia)]
- Tempos médio de incubação e infecção: $T_{\mathrm{inc}}=5,2\text{ dias}$ e $T_{\mathrm{inf}}=2,9\text{ dias}$ [[6](#referencia)]
- Demanda dos leitos de UTI: estimava-se que $3\%$ dos doentes precisariam de UTI [[3,5](#referencia)]
- Medianda dos tempos de UTI: $7$ dias [[3,5](#referencia)]
- Disponibilidade de leitos de UTI em SP em 2019: $27$ por $100$ mil habitantes [[4](#referencia)]
- População de São Paulo: $44,04$ milhões
- Dados iniciais:              
$$ \begin{bmatrix}
\mathcal{S}_0 \\
\mathcal{E}_0 \\
\mathcal{I}_0 \\
\mathcal{R}_0
\end{bmatrix}  =\begin{bmatrix}
0,999999 \\
10^{-6} \\
0 \\
0 
\end{bmatrix}  $$



O primeiro passo é compilar **todas** as células. Após isso, basta alterar os inputs seguindo as instruções de cada um:


*   ```Default```: se estiver **marcado**, vai escolher os **valores padrão** escritos no enunciado do problema. Mesmo se as outras variáveis estiverem diferentes, os gráficos gerados serão os padrões.

*   ```r_0```: slider para variar os valores da taxa de reprodução basal ($r_0$). Só funciona se a opção ```Default``` estiver **desmarcada**. **Importante**: se ```Default``` estiver **desmarcado**, mas se ```Random``` estiver **marcado**, então a taxa vai ser aleatória e não vai ser o valor do slider.

*   ```Random```: se ```Default``` estiver  **desmarcado** e se ```Random``` estiver **marcado**, então a taxa de reprodução basal ($r_0$) será **aleatória** (para mais detalhes, clique neste [link](#cell-id)). Esta opção é interessante para simular uma variação na taxa, além de conseguir oscilações nos valores de forma a simular uma possível segunda onda da doença.

* ```T_inc```: slider para variar os valores do tempo médio de incubação ($T_{\mathrm{inc}}$). Só funciona se a opção ```Default``` estiver **desmarcada**.

* ```T_inf```: slider para variar os valores do tempo médio de infecção ($T_{\mathrm{inf}}$). Só funciona se a opção ```Default``` estiver **desmarcada**.

* ```Porcentagem_UTI```: slider para variar os valores da porcetagem média da população de doentes que precisaria de UTI. Os valores vão de 0 a 0.1, ou seja, de 0% a 10%. Só funciona se a opção ```Default``` estiver **desmarcada**.

* ```Tempo_UTI```: slider para variar os valores da mediana de tempo que os doentes ficariam na UTI. Os valores vão de 0 a 10 dias. Só funciona se a opção ```Default``` estiver **desmarcada**.

Feito isso, o código vai ser compilado automaticamente com cada mudança nos inputs (sliders e checkbox). O resultado são dois gráficos, um com os dados de proporção populacional e outro com os dados de população absoluta junto com o total aproximado de pessoas que ficarão sem UTI. Os gráficos são interativos, pode-se dar zoom, selecionar as curvas e até mesmo exportar em png.

**Obs**: para visualizar o código usado é só clicar no botão ```Mostrar código```.


In [54]:
#@title Escolha os valores iniciais { run: "auto", display-mode: "form" }
Default = True #@param {type:"boolean"}
r_0 = 2.5 #@param {type:"slider", min:0.1, max:10, step:0.05,value:2.5}
Random = False #@param {type:"boolean"}
T_inc = 5.2 #@param {type:"slider", min:0.1, max:10, step:0.1}
T_inf = 2.9 #@param {type:"slider", min:0.1, max:10, step:0.1}
Porcentagem_UTI = 0.03 #@param {type:"slider", min:0, max:0.1, step:0.01}
Tempo_UTI = 7 #@param {type:"slider", min:0, max:10, step:1}

###################### Bibliotecas importadas #####################

import ipywidgets as widgets
import plotly.graph_objects as go
from IPython.display import display
from IPython.display import HTML
import numpy as np

###################### Dados iniciais #####################

if Default: # dados do enunciado
  r_0=2.5 #número de reprodução basal
  T_inc=5.2 #tempo de incubação
  T_inf=2.9 #tempo de infecção
  Porcentagem_UTI=0.03
  Tempo_UTI=7


tf=498 #número final de dias da simulação
dt=1 # passo (1 dia)
t=np.arange(0,tf+0.5,dt) # vetor com todos os dias até o 498º dia
n=len(t) #tamanho do vetor t => número de dias da simulação

S = np.ones(n)*0.999999 # vetor preenchido com as condições iniciais de S (suscetiveis)
E = np.ones(n)*(10**-6) # vetor preenchido com as condições iniciais de E (expostos) 
I = np.zeros(n) # vetor preenchido com as condições iniciais de I (infectados)
R = np.zeros(n) # vetor preenchido com as condições iniciais de R (recuperados)

def f(y):
    # função que recebe como argumento uma lista com 
    # 4 valores (S, E, I e R, respectivamente) e retorna 
    # a matriz f' do método do ponto médio
    a = (r_0/T_inf)*y[0]*y[2]
    b = (1/T_inc)*y[1]
    c = (1/T_inf)*y[2]
    S_ = -a
    E_ = (a - b)
    I_ = (b - c)
    R_ = c
    return [S_,E_,I_,R_]

for i in range(1,n):
    # loop para aplicação do método do ponto médio
    if Random == True and Default == False:
      a=np.random.rand(1)[0]*5
      r_0=abs(a*np.cos(2.5*np.pi*t[i]/498))+0.5 
    y05 = f([S[i-1],E[i-1],I[i-1],R[i-2]]) # chama a função f'
    S_05 = S[i-1] + y05[0]*(dt/2) #variável auxiliar do ponto médio
    E_05 = E[i-1] + y05[1]*(dt/2) #variável auxiliar do ponto médio
    I_05 = I[i-1] + y05[2]*(dt/2) #variável auxiliar do ponto médio
    R_05 = R[i-1] + y05[3]*(dt/2) #variável auxiliar do ponto médio
    y_ = [S_05,E_05,I_05,R_05] # cria a lista para depois jogar na função f'
    S[i] = S[i-1] + dt*f(y_)[0] # método de euler com o ponto médio
    E[i] = E[i-1] + dt*f(y_)[1] # método de euler com o ponto médio
    I[i] = I[i-1] + dt*f(y_)[2] # método de euler com o ponto médio
    R[i] = R[i-1] + dt*f(y_)[3] # método de euler com o ponto médio

###################### Transformando em população absoluta #####################

S0 = np.ones(n) # vetor auxiliar suscetiveis
E0 = np.ones(n) # vetor auxiliar expostos
I0 = np.zeros(n) # vetor auxiliar infectados
R0 = np.zeros(n) # vetor auxiliar recuperados
L0 = np.zeros(n) # vetor auxiliar da quantidade de leitos total em SP
F0 = np.zeros(n) # vetor auxiliar do fluxo de infectados para recuperados
N0 = np.zeros(n) # vetor auxiliar da quantidade de necessitados de UTI

for j in range(n):
    # loop para multiplicar a proporção pela população de SP
    S0[j] = int(S[j]*44040000)
    E0[j] = int(E[j]*44040000)
    I0[j] = int(I[j]*44040000)
    R0[j] = int(R[j]*44040000)
    L0[j] = int((27*44040000)/100000)
    if j>=1:
        # calcula o fluxo de infectados para recuperados
        F0[j] = int(abs(R0[j-1]-R0[j])*Porcentagem_UTI) 
    

for p in range(n):
    # loop para calcular a quantidade de pessoas que precisam de UTI
    # baseado no número de doentes 
    s=0
    for l in range(Tempo_UTI):
      if l<=p:
        s=s+F0[p-l]
    N0[p]=s

pessoas_sem_uti = [] # lista da quantidade de pessoas que ficaria sem leitos
for b in range(len(L0)):
    # loop para calcular a diferença entre necessitados de leitos
    # e a quantidade total de leitos
    if N0[b]>=L0[b]:
        pessoas_sem_uti.append(N0[b]-L0[b])

total_sem_uti = sum(pessoas_sem_uti) # quantidade aproximada de pessoas sem leitos

# Layout Grafico 1
layout = go.Layout(
    title="Proporção do modelo SEIR versus tempo",
    title_x=0.5,
    xaxis=dict(
        title="Tempo (dias)"
        ),
        yaxis=dict(
        title="Fração da população de SP"
        ) 
)
fig = go.Figure(layout = layout)

# Adiciona cada gráfico de linha com marcadores
fig.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=S,
        name='Suscetíveis',
        marker=dict(
            color='darkturquoise',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)
fig.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=E,
        name='Expostos',
        marker=dict(
            color='darkmagenta',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)
fig.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=I,
        name='Infectados',
        marker=dict(
            color='red',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)
fig.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=R,
        name='Recuperados',
        marker=dict(
            color='yellow',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)

# Layout Grafico 2

layout1 = go.Layout(
    title="População de SP versus tempo",
    title_x=0.5,
    xaxis=dict(
        title="Tempo (dias)"
        ),
        yaxis=dict(
        title="População"
        ),
    annotations=[
            go.layout.Annotation(
                text=f"O número aproximado<br> de pessoas que ficarão<br> sem UTI é <b>{total_sem_uti:,.0f}<b>",
                align='left',
                showarrow=False,
                xref='paper',
                yref='paper',
                x=1.17,
                y=0,
                bordercolor='rgb(42, 63, 126)',
                borderwidth=1
            )
        ]
)

# Adiciona cada gráfico de linha com marcadores

fig1 = go.Figure(layout = layout1)

fig1.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=S0,
        name='Suscetíveis',
        marker=dict(
            color='darkturquoise',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)
fig1.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=E0,
        name='Expostos',
        marker=dict(
            color='darkmagenta',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)
fig1.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=I0,
        name='Infectados',
        marker=dict(
            color='red',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)
fig1.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=R0,
        name='Recuperados',
        marker=dict(
            color='yellow',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)
fig1.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=L0,
        name='Leitos',
        marker=dict(
            color='orange',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)
fig1.add_trace(
    go.Scatter(
        mode='lines+markers',
        x=t,
        y=N0,
        name='Necessidade de Leitos',
        marker=dict(
            color='green',
            size=6,
            opacity=0.4,
            line=dict(
                color='burlywood',
                width=1
            )
        ),
        showlegend=True
    )
)

# Plota os gráficos 1 e 2

display(fig, fig1)



# Resolução pelo Método do Ponto Médio

Seja $f(t,Y(t))$ a matriz $4\times1$ tal que:
$$f(t,Y(t))=\begin{bmatrix}
\frac{d \mathcal{S}}{d t} \\
\frac{d \mathcal{E}}{d t} \\
\frac{d \mathcal{I}}{d t} \\
\frac{d \mathcal{R}}{d t}
\end{bmatrix}=\begin{bmatrix}-\frac{r_{0}}{T_{\mathrm{inf}}} \mathcal{S} \mathcal{I} \\
\frac{r_{0}}{T_{\mathrm{inf}}} \mathcal{S} \mathcal{I}-\frac{1}{T_{\mathrm{inc}}} \mathcal{E} \\
\frac{1}{T_{\mathrm{inc}}} \mathcal{E}-\frac{1}{T_{\mathrm{inf}}} \mathcal{I} \\
\frac{1}{T_{\mathrm{inf}}} \mathcal{I}
\end{bmatrix}$$
E seja $Y(t)$:
$$Y(t)=\begin{bmatrix}
\mathcal{S(t)} \\
\mathcal{E(t)} \\
\mathcal{I(t)} \\
\mathcal{R(t)}
\end{bmatrix}$$
O Método do Ponto Pédio é uma versão melhorada do Método de Euler para resolução de equações diferencias. Ele consiste em etapas iterativas que seguem a seguinte expressão:
$$Y_{i+\frac{1}{2}}=Y_i+\dfrac{h}{2}F(t,Y_i)\\ 
Y_{i+1}=Y_i+hF(t_{i+\frac{1}{2}},Y_{i+\frac{1}{2}})$$
Sendo que que $h$ é o passo de cada iteração, no nosso caso o passo é de $1$ dia. O objetivo final é chegar a $Y(t=498)$ e analisar graficamente os valores.

# Discussões Obrigatórias


### 1. Qual a maior proporção de pessoas infecciosas observadas em um único dia? Considerando que o estado de São Paulo possui 44,04 milhões de habitantes, essa proporção representa quantas pessoas doentes ao mesmo em um estado como o nosso? Quantos dias são necessários para chegar no pico de doentes?
   Considerando o primeiro gráfico acima, ao estudar a curva referente às pessoas infecciosas, a maior proporção de pessoas doentes observadas em um único dia é de aproximadamente 8,25%, pico que ocorreria no dia 99 de pandemia. Para observar qual seria o equivalente a essa proporção em um estado como São Paulo, plotamos o segundo gráfico acima multiplicando as proporções pelo número de habitantes (44,04 milhões) e encontramos, analisando a curva de infecciosos, que o pico aproximado é de 3,63 milhões de infectados, um valor alarmante para a saúde da população.


### 2.   Estime o número de leitos de UTI para lidar com o pico. O número de leitos disponíveis é suficiente para lidar com o pico de demanda? Qual o excesso de demanda máximo, se houver?

Pelo segundo gráfico acima, observamos que o número total de leitos disponíveis no estado de São Paulo seria de 11,89 mil (usando as informações de 2019 em que haveria 27 leitos para cada 100 mil habitantes). Enquanto que seriam necessários um total de 258,954 mil leitos para lidar com o pico da demanda, que ocorreria no dia 102 de pandemia, apenas alguns dias após o pico de pessoas doentes. Nesse sentido, haveria um excesso de  247,064 mil pessoas necessitando de leitos nos hospitais neste dia de demanda máxima.


### 3.   Você consegue estimar quantas pessoas ao todo iriam ficar sem acesso a UTIs? Justifique a forma que você fez essa estimativa.
Para estimar o número de pessoas ao todo que iriam ficar sem acesso a UTIs, deveríamos integrar a função necessidade de leitos ao longo dos valores em que esta função supera a capacidade de leitos do estado de São Paulo. Feito esta integral, deveria subtrair a área retangular da curva Leitos em São Paulo no mesmo período.

Contudo, considerando que já tínhamos os valores por dia de pessoas necessitando de leitos, bastava subtrair este valor diário pela quantidade de leitos disponíveis ao todo. Isso só é feito para quando o número da necessidade de leitos fosse superior ao número de leitos disponíveis em São Paulo.
Após, somando essas diferenças, obtém-se uma aproximação da quantidade de pessoas que ficariam sem acesso a UTIs. Dito isso, encontramos um valor de 7.229.088 pessoas no total sem esse acesso a UTIs.

 ### 4.  Com base nessa simulação o que você diria ao governador do estado no início da pandemia? As restrições de circulação foram necessárias?
Notamos que a situação causada pela Covid-19 é de causar preocupação, pois o estado de São Paulo não estaria preparado para atender a demanda hospitalar visto o número alarmante de pessoas doentes necessitando do acesso a UTIs e de cuidados. Isso sem contar que o número de UTIs que o estado tinha não era somente para Covid-19, mas também para outras doenças, o que reduziria ainda mais a quantidade acessível para os infectados pelo SARS-CoV-2. 

Além disso, a doença causada pela Covid-19, por ser imprevisível e sem ter tratamento previamente estudado e consolidado, deve ser avaliada com muita cautela e precaução. Dito isso, para conter o avanço da pandemia em seu início, recomendaria ao governador do estado restringir a circulação de pessoas, limitando a quantidade de pessoas em determinados estabelecimentos e instituições, incentivar o uso de máscaras e distanciamento social, além de realizar campanhas de conscientização sobre o Coronavírus a fim de convencer a população que as medidas tomadas serão de intrínseca importância para o futuro, evitando milhares de mortes.


Fica claro que se essas medidas tivessem sido adotadas poderíamos ter retardado o pico da doença. Com este tempo a mais, o sistema de saúde teria mais chances de se organizar a aumentar a capacidade de leitos, de forma a garantir que menos pessoas ficassem sem leitos e fossem atendidas. Isso, além de destinar mais tempo para pesquisas e busca de vacinas. Sem contar que o número de infectados ao mesmo tempo no pico poderia reduzir. 

### 5.  Alterando as variáveis

Um ponto interessante de ter essa simulação numérica é o fato de poder alterar os valores iniciais e conseguir prever as mudanças finais devido a certos parâmetros. Começando com a taxa de reprodução basal ($r_0$). Ao aumentar o seu valor (de 2,5 para um valor superior) sem alterar os outros parâmetros, percebe-se um aumento no número de pessoas infectadas (e no número de expostas), além do aumento de pessoas que ficariam sem leitos. Além disso, é possível notar que o pico acontece muito mais cedo, o que é perigoso para o sistema público de saúde. Já se este valor for reduzido, é possível perceber que o número de pessoas infectadas no pico e que a necessidade de leitos reduzem, sendo que o pico acontece muito mais tardamente.

Desta forma, constatamos que a diminuição de $r_0$ proporciona um intervalo de tempo muito maior entre o início da pandemia e o pico de infectados, reduzindo a necessidade de leitos na UTI para os doentes durante a pandemia, ao passo que o aumento dessa variável provoca um adiantamento drástico do pico de infectados, amplificando a demanda por leitos e agravando as condições dos hospitais para tratar seus pacientes. 

Ao variarmos o tempo médio de infecção ($T_{inf}$) de 2,9 para 4 dias, observamos um aumento surpreendente no número pico de pessoas infectados pelo Covid-19, saltando de 3,93 milhões para 4,40 milhões, enquanto se alterarmos de 2,9 para 1,5,  esse pico diminui,  afetando 2,28 milhões de indivíduos. Ou seja, com o aumento desta variável há um aumento no número de pessoas infectadas no pico e com a sua diminuição há uma redução no número de indivíduos.
 
Continuando esses testes, também fica claro o impacto da mudança da porcentagem da demanda de leitos de UTI para os infectados com Covid-19, a qual, com sua diminuição de 3% para 1%, possibilita a redução da necessidade de leitos de 7,2 milhões para 1,9 milhões, uma extrema diferença, o que aliviaria a capacidade de tratamento dos hospitais e poderia evitar inúmeras mortes durante a pandemia.

Por fim, analisando a variação do tempo médio de uso dos leitos de UTI pelos infectados de Covid-19, caso ocorra o aumento de, por exemplo, 7 para 9 dias, observamos um crescimento assustador em 2 milhões de pacientes que ficariam sem leitos (9 milhões no total), enquanto a diminuição de 7 para 5 dias, temos o efeito oposto, com o decrescimento de 2 milhões de de pacientes, com cerca de 5 milhões de infectados sem leitos.

Assim, concluí-se que cada variável considerada nos cálculos tem um impacto relevante no desenvolvimento dos casos na pandemia e também na estrutura necessária para o tratamento de pacientes nos hospitais do país, sendo vital o seu estudo para que os efeitos da pandemia na população sejam minimizadas, o que resultaria em um menor contágio e menos infectados e, consequentemente, menos óbitos.   


<a name="cell-id"></a>
# $r_0$ Aleatório

Numa situação mais "realista", a taxa de reprodução basal é um valor que varia dependendo da situação da pandemia e do comportamento populacional. Por exemplo, supondo que a população não realize as medidas de distanciamento social e de segurança, é esperado que a taxa aumente, mas que este crescimento tenha algumas oscilações. Desta forma, é uma hipótese meio forte argumentar que o valor é constante para todo o período. 

Neste sentido, com o intuito de simular uma variação mais "realista", ou seja, com variações, procurou se expressar $r_0$ como uma função dependente do tempo. A função pensada foi:     
$$r_0(t)=\left|a\cos{\left(\dfrac{2.5\cdot\pi\cdot t}{498}\right)}\right|+0.5$$
Sendo que $t$ é o tempo em dias e $a$ é um número aleatório (gerado automaticamente e aleatóriamente no código) entre 0 e 5 (basicamente ele indica a amplitude da função de forma que $0.5\le r_0 \le 5.5$).

A escolha da função foi pensando em gerar oscilações de $r_0$, por isso uma função cosseno, pois é razoável pensar que este fator oscila com o tempo. Além disso, o fator aleatório garante valores diferentes de $r_0$ com o passar do tempo. Ademais, observando o gráfico na célula abaixo, percebe-se que a função tem alguns picos e isso foi escolhido de propósito para simluar os momentos em que a população não tomava mais os cuidados sanitários (em feriados por exemplo) e a taxa atingia um valor acima do padrão.

Por fim, esta implementação acaba gerando gráficos com algumas oscilações e até com duas ondas de infecção. Isso, apesar de ser algo opcional que o grupo decidiu trazar, é interessante de se observar, ainda mais considerando que em muitos locais foi exatamente o que aconteceu em relação à pandemia

In [ ]:
from IPython.display import IFrame
IFrame(src="https://www.desmos.com/calculator/dvzwm8quts", width=700, height=375)

<a name="referencia"></a>

# Referências

[1] Julie C. Blackwood and Lauren M. Childs. An introduction to compartmental modeling for the budding infectious disease modeler. Letters in Biomathematics, $5(1): 195-221$, December 2018 .

[2] Ying Liu, Albert A Gayle, Annelies Wilder-Smith, and Joacim Rocklöv. The reproductive number of COVID- 19 is higher compared to SARS coronavirus. Journal of Travel Medicine, 27(2), March $2020 .$

[3] Seyed M. Moghadas, Affan Shoukat, Meagan C. Fitzpatrick, Chad R. Wells, Pratha Sah, Abhishek Pandey, Jeffrey D. Sachs, Zheng Wang, Lauren A. Meyers, Burton H. Singer, and Alison P. Galvani. Projecting hospital utilization during the COVID-19 outbreaks in the United States. Proceedings of the National Academy of Sciences, $117(16): 9122-9126$, April 2020 .

[4] Úrsula Neves. Covid-19: IBGE divulga distribuição de UTIs, respiradores e profissionais de saúde no Brasil. https://pebmed.com.br/covid-19-ibgedivulga-distribuicao-de-utis-respiradores-e-profissionais-de-saude-no-brasil/, May $2020 .$

[5] Rachel Lane Socolovithc, Renata Rego Lins Fumis, Bruno Martins Tomazini, Laerte Pastore, Filomena Regina Barbosa Gomes Galas, Luciano Cesar Pontes de Azevedo, and Eduardo Leite Vieira Costa. Epidemiology, outcomes, and the use of intensive care unit resources of critically ill patients diagnosed with COVID-19 in Sao Paulo, Brazil: A cohort study. PLOS ONE, 15 (12):e0243269, December 2020 .

[6] Joseph T Wu, Kathy Leung, and Gabriel M Leung. Nowcasting and forecasting the potential domestic and international spread of the 2019-nCoV outbreak originating in Wuhan, China: A modelling study. The Lancet, $395(10225): 689-697$, February 2020 .
